# IIC2413 - Control 2

SQL avanzado, diseño de bases de datos e índices.

La entrega es subiendo la solución en este mismo archivo a Canvas. Fecha: Viernes 8 de octubre a las 20:00

## Introduccion

Has sido contratado/a por la famosa empresa de video juegos _Nientiendo_ para trabajar en la implementación del nuevo juego de su exitosa franquicia _Bolsimon_, titulado _Bolsimon Unión_, el cual consiste en combinar los ya queridos monstruos de bolsillo de la franquicia con el popular género de [_Multiplayer online battle arenas_ (MOBA)](https://en.wikipedia.org/wiki/Multiplayer_online_battle_arena). El juego es de carácter casual y será lanzado en Android, iOS y en la consola _Nientiendo Swap_.

Una partida de Bolsimon Unión consiste en dos equipos de 5 jugadores (azul y rojo) que se enfrentan durante 10 minutos con el objetivo de conseguir la mayor cantidad de puntos posible. Las partidas se caracterizan por lo siguiente:

* Cada jugador elige un bolsimon de los que tenga disponibles en su _pool_ de bolsimones. Un bolsimon no puede estar dos veces en una misma partida.
* A lo largo de la partida los jugadores deben dejar fuera de combate (KO) a los bolsimones oponentes para avanzar hacia su base y así poder obtener puntos (_score_).
* Cuando más de un jugador participa en dejar KO a un oponente, aquellos que no dieron el último golpe reciben una asistencia.
* Un bolsimon KO vuelve a la partida luego de unos segundos.
* El equipo ganador es aquel que logró obtener más puntos luego de 10 minutos de partida.
* Para las estadísticas el juego guarda el detalle de cada KO y cada _score_.

Como el nuevo ingeniero estrella del proyecto se te dejó a cargo de escribir algunas consultas para la base de datos PostgreSQL del servidor de Bolsimon Unión y de  mejorar el diseño de la base de datos para soportar nuevas funcionalidades en el juego. Evidentemente debes implementar aquellas mejoras en el _DBMS_ tu mismo.

## I. Consultas SQL (16 pts)

El esquema actual de la base de datos es el siguiente:

* `usuario(id int, email varchar(100), username varchar(100), rating int)`
* `bolsimon(id int, nombre varchar(30), tipo varchar(30))`
* `partida(id int, fecha_inicio date, estado varchar)`
* `usuario_partida_bolsimon(id int, id_jugador int, id_bolsimon int, id_partida int , equipo varchar(5))`
* `ko(id int, id_jugador int, id_victima int, id_partida int, timestamp float, segundos_reaparecer int)`
* `asistencia(id int, id_jugador int, id_ko int)`
* `score(id int, id_partida int, id_jugador int, puntaje int, timestamp float)`

Ejecuta las siguientes celda para descargar la base de datos a tu máquina en Colab y montarla en Postgres:

In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
!pip install gdown
!gdown https://drive.google.com/u/1/uc?id=1RqaKncDfYYfPMYckbM9Ui4YSkM_FsB1O
!sudo -u postgres psql -c "CREATE SCHEMA postgres"
!psql postgres < /content/database.sql

In [ ]:
%%sql
ALTER TABLE usuario ADD PRIMARY KEY (id);
ALTER TABLE bolsimon ADD PRIMARY KEY (id);
ALTER TABLE partida ADD PRIMARY KEY (id);
ALTER TABLE usuario_partida_bolsimon ADD PRIMARY KEY (id);
ALTER TABLE ko ADD PRIMARY KEY (id);
ALTER TABLE asistencia ADD PRIMARY KEY (id);
ALTER TABLE score ADD PRIMARY KEY (id);

Si la siguiente consulta funciona bien entonces estas listo para empezar.

In [ ]:
%%sql
SELECT * FROM bolsimon LIMIT 20;

Escribe las siguientes consultas necesarias para el funcionamiento de Bolsimon Unión:

Pro Tips:
* La sentencia [`LIMIT`](https://www.postgresql.org/docs/8.1/queries-limit.html) 
que limita los resultados de una consulta es necesaria para algunas consultas.
* Algunas de las consultas pueden ser complejas por lo que la sentencia [`WITH`](https://www.postgresql.org/docs/8.1/queries-limit.html) que permite "renombrar" una consulta, podría ser útil para mejorar la legibilidad de tus consultas y disminuir la duplicacion.
* Para transformar valores nulos a algún valor que nos interese, se puede usar `COALESCE(<valor potencialmente nulo>, <otro valor>)`, por ejemplo, `SELECT COALESCE(age, 50)` para considerar que todos quienes no tengan edad especificada se asuman de 50 años.

1. Para el usuario de email `elizabeth@cooper.net` encontrar la cantidad total de KOs, Asistencias y puntaje obtenidos en todas sus partidas.

In [ ]:
%%sql

2. Una consulta que entregue el equipo ganador de la partida de id `12345`

In [ ]:
%%sql

3. Encontrar los usernames  y ratings de los 5 usuarios que han usado el  Bolsimon  `Pikecho` en más partidas.


In [ ]:
%%sql

4. Para la partida de id `256` mostrar el tiempo en segundos que cada bolsimon pasó fuera de combate.



In [ ]:
%%sql

5. Los ids de la pareja de jugadores que más asistencias se hayan hecho mutuamente. (Es decir si `A` asistió a `B` 5 veces y `B` a `A` 3 veces el total es 8)

In [ ]:
%%sql

6. Encontrar el username del usuario que jugó más partidas en un día. Tip: Con la sentencia `date_trunc('day', <fecha>)` se puede extraer el día a partir de una fecha completa.

In [ ]:
%%sql

7. Encontrar el Bolsimon, equipo, la cantidad KOs realizados y puntos totales de cada jugador de la partida de ID `32`. Los jugadores del equipo ganador deben aparecer primero y los jugadores de cada equipo deben ir ordenados por puntos totales y KOs realizados de manera descendente.

In [ ]:
%%sql

8. Encontrar el nombre del Bolsimon de mejor y peor winrate (% de victorias).Muestre los winrates de cada uno (basta con mostrar 2 filas en que una corresponde al mejor y otra al peor).

**Protips**:
- Para contar la cantidad de `true` en una columna (o agregación) hay que castear los booleanos a entero (`1` o `0`) y despues sumar, algo así:
`SUM(CAST(<expresion o columna booleana> AS INTEGER)`.
- Para hacer división con decimales en postgres es necesario que al menos uno de los dos términos sea de tipo `FLOAT`, es decir podemos hacer `SELECT CAST(1 AS FLOAT) / 2` y eso da `0.5`.

In [ ]:
%%sql

## II. Aumentar el diseño de la base de datos. (8 pts)

Evidentemente el esquema actual no soporta todas las funcionalidades necesarias para que el juego se convierta en el éxito que el mercado espera. La funcionalidad más importante que se debe implementar ahora es que cada bolsimon pueda tener sus propios ataques (ej: _el impactrueno_).

Los ataques tienen nombre y nivel (básico, intermedio y avanzado).

Cada bolsimon tiene 3 pares de ataques (1 par de cada nivel.). Dentro de una partida un jugador debe elegir un ataque de cada par. Notar que los ataques no son necesariamente únicos de un solo Bolsimon.

1. Agrega al esquema anterior las tablas  y restricciones de integridad que creas necesarias para modelar la nueva funcionalidad. Escribe y ejecuta las sentencias SQL necesarias para implementar tu esquema en la base de datos.

2. Justifica que tu esquema está bien diseñado.

*Responde acá*

In [ ]:
%%sql 

## III. Índices (5 pts)

Se espera que Bolsimon Unión reciba un alto flujo de jugadores y por lo tanto habran millones de tuplas en las tablas de la base de datos. Como ingeniero encargado de la DB es tu responsabilidad que esta funcione correctamente aunque las tablas esten altamente pobladas. Sugiere al menos 5 índices que creas sean los más necesarios y justifica por qué. Además escribe y ejecuta las sentencias SQL necesarias para modificar la base de datos.

*Responde acá*

In [ ]:
%%sql